In [53]:
# For vscode
import pandas as pd #reading dataset
import matplotlib as mpl    #plotting, visualization
import matplotlib.pyplot as plt
import tensorflow as tf #building model
from tensorflow.keras.models import Model
import numpy as np  #playing output with model
from sklearn.model_selection import train_test_split    #model selection and scaling data
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

import sys
sys.path.append('../../../')   # Add parent directory to Python path
from utils.Plots import *
RANDOM_SEED = 42

df = pd.read_csv('../../../data/handlebar/processed_segments_overlap_50.csv') 
df.drop('Unnamed: 0', axis=1, inplace=True)
df


,curb_scene,Acc-Z_1,Acc-Z_2,Acc-Z_3,Acc-Z_4,Acc-Z_5,Acc-Z_6,Acc-Z_7,Acc-Z_8,Acc-Z_9,...,Acc-Z_91,Acc-Z_92,Acc-Z_93,Acc-Z_94,Acc-Z_95,Acc-Z_96,Acc-Z_97,Acc-Z_98,Acc-Z_99,Acc-Z_100
0,0.0,8.762772,8.753204,8.762772,8.781937,8.753204,8.748413,8.738831,8.748413,8.743622,...,8.786728,8.801086,8.820236,8.786728,8.777145,8.772354,8.781937,8.801086,8.781937,8.753204
1,0.0,8.762772,8.762772,8.748413,8.767563,8.729263,8.695740,8.724472,8.767563,8.767563,...,8.753204,8.772354,8.786728,8.781937,8.786728,8.777145,8.738831,8.820236,8.738831,8.757996
2,0.0,8.781937,8.796295,8.772354,8.772354,8.772354,8.786728,8.781937,8.801086,8.767563,...,8.777145,8.753204,8.767563,8.762772,8.762772,8.753204,8.748413,8.762772,8.772354,8.767563
3,0.0,8.781937,8.753204,8.801086,8.753204,8.829819,8.753204,8.786728,8.815445,8.762772,...,8.805878,8.796295,8.781937,8.801086,8.810669,8.796295,8.805878,8.791504,8.791504,8.772354
4,0.0,8.757996,8.762772,8.777145,8.767563,8.786728,8.777145,8.753204,8.757996,8.777145,...,8.757996,8.757996,8.772354,8.767563,8.757996,8.743622,8.777145,8.743622,8.786728,8.796295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9312,1.0,-1.575378,5.295975,5.295975,16.682800,19.517532,14.781799,14.609421,7.469910,11.252747,...,-6.689407,2.274490,4.649536,-18.933350,-24.708160,-21.212631,-3.002335,10.764328,19.019547,19.871872
9313,1.0,0.100555,-6.871353,3.026276,17.348389,15.734695,13.632584,13.632584,14.556747,12.014099,...,-22.385788,-30.980972,-23.218964,-6.301544,14.973343,18.669983,26.542114,38.690308,48.295837,53.400284
9314,1.0,6.004654,9.461884,15.811310,22.572540,20.719421,20.719421,7.910446,0.928955,-0.574615,...,4.922485,19.182343,20.283676,20.283676,12.129028,-3.567368,-3.567368,1.077393,9.370911,10.749970
9315,1.0,60.429657,25.072083,9.495407,-14.068329,-19.512741,-11.065994,2.049438,2.049438,10.376465,...,0.210693,0.804459,0.804459,11.267120,10.807434,8.063675,4.228165,4.505890,19.072205,19.072205


In [45]:
# For Colab
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from datetime import datetime
from matplotlib import pyplot as plt
from matplotlib import dates as md
from sklearn.model_selection import train_test_split    #model selection and scaling data
from sklearn.preprocessing import StandardScaler
RANDOM_SEED = 42

import os
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Project/data/processed_segments_overlap_50.csv')
#df.drop('Unnamed: 0', axis=1, inplace=True)
# # 0 for normal, 1 for abnormal
df['curb_scene'].value_counts()
df.head()

ModuleNotFoundError: No module named 'google.colab'

In [51]:
# Suppose df is your DataFrame, and df['curb_scene'] is your label column
train_data, test_data, train_labels, test_labels = train_test_split(df, df['curb_scene'], 
                                                                    test_size=0.2, 
                                                                    random_state=RANDOM_SEED)
                                                        


In [52]:
#scaling data, because neural network works better with scaled data
scaler = StandardScaler()
data_scaled = scaler.fit(train_data.iloc[:, 3:])
#transform data
train_data_scaled = scaler.transform(train_data.iloc[:, 3:])
test_data_scaled = scaler.transform(test_data.iloc[:, 3:])


In [54]:
# Convert the scaled data back to DataFrames for further processing
train_data_scaled_df = pd.DataFrame(train_data_scaled, columns=train_data.columns[3:])
test_data_scaled_df = pd.DataFrame(test_data_scaled, columns=test_data.columns[3:])
# Add the 'curb_scene', 'start_time', and 'end_time' columns back to the scaled DataFrames
train_data_scaled_df['curb_scene'] = train_data['curb_scene'].values
test_data_scaled_df['curb_scene'] = test_data['curb_scene'].values

In [55]:
# Filter and slice the data
normal_train_data_scaled = train_data_scaled_df[train_data_scaled_df['curb_scene'] == 0.0].iloc[:, :-3]
abnormal_train_data_scaled = train_data_scaled_df[train_data_scaled_df['curb_scene'] == 1.0].iloc[:, :-3]
normal_test_data_scaled = test_data_scaled_df[test_data_scaled_df['curb_scene'] == 0.0].iloc[:, :-3]
abnormal_test_data_scaled = test_data_scaled_df[test_data_scaled_df['curb_scene'] == 1.0].iloc[:, :-3]
# Example usage: Accessing the original index
print(normal_test_data_scaled.index)
print(abnormal_test_data_scaled.index)

Index([   0,    1,    2,    3,    4,    5,    6,    8,    9,   10,
       ...
       1854, 1855, 1856, 1857, 1858, 1859, 1860, 1861, 1862, 1863],
      dtype='int64', length=1796)
Index([   7,   54,   63,   73,   92,   94,  119,  188,  204,  209,  218,  330,
        358,  363,  402,  412,  421,  456,  530,  553,  567,  584,  590,  597,
        757,  784,  796,  840,  891,  894,  949,  950,  955,  962,  978,  979,
        990, 1006, 1033, 1038, 1071, 1126, 1135, 1143, 1168, 1195, 1266, 1272,
       1274, 1275, 1279, 1359, 1384, 1399, 1409, 1442, 1462, 1472, 1520, 1521,
       1562, 1581, 1598, 1624, 1628, 1655, 1673, 1763],
      dtype='int64')


In [58]:
# Cast to float32
normal_train_data_scaled = normal_train_data_scaled.astype("float32")
abnormal_train_data_scaled = abnormal_train_data_scaled.astype("float32")
normal_test_data_scaled = normal_test_data_scaled.astype("float32")
abnormal_test_data_scaled = abnormal_test_data_scaled.astype("float32")

In [59]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout
import keras_tuner as kt

class AutoEncoder(Model):
    def __init__(self, hp):
        super(AutoEncoder, self).__init__()
        
        # Encoder
        self.encoder = tf.keras.Sequential()
        for i in range(hp.Int('num_encoder_layers', 1, 3)):
            self.encoder.add(Dense(units=hp.Int(f'units_encoder_{i}', min_value=32, max_value=128, step=32), activation='tanh'))
            self.encoder.add(Dropout(rate=hp.Float(f'dropout_encoder_{i}', min_value=0.1, max_value=0.5, step=0.1)))
        
        # Decoder
        self.decoder = tf.keras.Sequential()
        for i in range(hp.Int('num_decoder_layers', 1, 3)):
            self.decoder.add(Dense(units=hp.Int(f'units_decoder_{i}', min_value=32, max_value=128, step=32), activation='tanh'))
            self.decoder.add(Dropout(rate=hp.Float(f'dropout_decoder_{i}', min_value=0.1, max_value=0.5, step=0.1)))
        
        # Output layer
        self.decoder.add(Dense(100, activation='tanh'))

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [60]:
def build_model(hp):
    model = AutoEncoder(hp)
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mae')
    return model

In [61]:
# Define the tuner
tuner = kt.RandomSearch(
    hypermodel = build_model,
    objective='val_loss',
    max_trials=3,  # Number of different hyperparameter combinations to try
    executions_per_trial=2,  # Number of models to build and fit for each trial
    directory='my_dir',
    project_name='autoencoder_tuning'
)

tuner.search_space_summary()

Reloading Tuner from my_dir\autoencoder_tuning\tuner0.json
Search space summary
Default search space size: 13
num_encoder_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_encoder_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
dropout_encoder_0 (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
num_decoder_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_decoder_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
dropout_decoder_0 (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampl

In [57]:
# Perform the search
tuner.search(
    normal_train_data_scaled, 
    normal_train_data_scaled,
    epochs=100,
    validation_split=0.3,
    batch_size=kt.HyperParameters().Int('batch_size', min_value=32, max_value=512, step=32),
)

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the best hyperparameters
print("Best hyperparameters:")
for key, value in best_hps.values.items():
    print(f"{key}: {value}")


The optimal number of encoder layers is 3.
The optimal number of units in each encoder layer are [32, 64, 32].
The optimal dropout rates for each encoder layer are [0.30000000000000004, 0.2, 0.1].
The optimal number of decoder layers is 1.
The optimal number of units in each decoder layer are [128].
The optimal dropout rates for each decoder layer are [0.30000000000000004].
The optimal learning rate is 0.0006621908905224919.

